<a href="https://colab.research.google.com/github/bduff4/GenAI/blob/main/preHW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Brennan Duff
* Generative AI
* 2/20/2026
* Assignment 3, Sliding Window Stress Test, the purpose of this assignment is to observe and report on how context window management and sliding window slicing affect the short-term memory of a chatbot.

In [1]:
# Install the needed libraries

!pip install -qU langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 127.3 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 1. Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=0.1
)

# 2. Initialize the History
# We start with the SystemMessage to set the persona
chat_history = [
    SystemMessage(content="You are a witty comedian who specializes in science jokes.") # Dictates how the chatbot should behave and respond.
]

print("--- Science Comedian Bot (Type 'exit' to stop) ---")

# 3. The Chat Loop
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Comedian: I'm 'splitting' like an atom! Goodbye!")
        break

    # Add user's message to history
    chat_history.append(HumanMessage(content=user_input))

    # Get the AI response
    # We use .text here as a shortcut for Gemini 3 to get clean text

    system_msg = chat_history[0] # set the SystemMessage as its own variable
    windowed_context = [system_msg] + chat_history[-2:] # pass the SystemMessage and only the last two messages to the context window
    response = llm.invoke(windowed_context) # call with only the windowed context + SystemMessage
    ai_text = response.content[0]['text'] # extract plain text

    print(f"Comedian: {ai_text}")

    # Add the AI's response to history so it remembers its own jokes!
    chat_history.append(AIMessage(content=ai_text))


--- Science Comedian Bot (Type 'exit' to stop) ---
You: the secret vault code is 4115
Comedian: 4115? Thanks for the tip! I’ll store that right next to my knowledge of the Krebs cycle—in a part of my brain that I can never seem to access when I’m under pressure.

But let’s look at the chemistry here. 41 is **Niobium** and 15 is **Phosphorus**. So your vault code is basically **NbP**. I assume that stands for "No Big Problems," or perhaps "Nuclear Biological Party." 

Either way, I hope whatever you’re hiding in there is more stable than a Francium atom in a bubble bath. Because if that vault is holding antimatter, 4115 isn't a code—it’s a countdown!
You: What is the capital of France?
Comedian: The capital of France is **Paris**! 

Though, if you’re a chemist, you might argue the *real* capital is **Francium** (Element 87). But I wouldn't recommend moving there; it has a half-life of about 22 minutes. You’d barely have time to order a baguette before the entire city underwent radioacti

# **Analysis**

Compare Trial A to Trial C. Why did the bot forget the vault code even though the message is still inside your chat_history list?

* The context window was set to cut off at the last 2 messages, so anything past said cutoff is exempt from the next prompt generated, sans the SystemMessage.

In Trial C, if the bot did not know the code, did it admit it, or did it try to guess (hallucinate)? Explain what happens when a chatbot has a window that is too small.

* Implicitly, the chat bot admitted it did not know the code, as it  hallucinated multiple different codes to compensate. When a context window is too small, models do not recieve sufficient conversation history. This can lead to: "forgetting" instructions, names, etc. established previously, repition of facts, jokes, etc. that it had already stated, and hallucination of facts with plausible information as demonstrated.

Explain the role of [system_msg] in your code. If we removed that part and only sent chat_history[-2:], how would the bot's behavior change?

* The System Message provides the persona of the chatbot, so if the [system_msg] was not passed to the context window, the bot would inevitably start breaking off of the preestablished character in later prompts. It would also allow the user to "hijack" the instructions more easily, ie. "stop being a comedian and stop refering to science related topics."  

What if we want a smarter bot that remembers more? Repeat the above and explain if a window of 6 messages solves the problem.

* A window of 6 naturally solves the issue, as I only called the bot a total of two times in trial B before I asked for the code. As such, the bot is "smarter". The same situation as the previous experiment will occur if you run the bot enough, as the original message will fall out of the window.

In [8]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 1. Setup
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.1
)

# 2. Initialize the History
# We start with the SystemMessage to set the persona
chat_history = [
    SystemMessage(content="You are a witty comedian who specializes in science jokes.")
]

print("--- Science Comedian Bot (Type 'exit' to stop) ---")

# 3. The Chat Loop
while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Comedian: I'm 'splitting' like an atom! Goodbye!")
        break

    # Add user's message to history
    chat_history.append(HumanMessage(content=user_input))

    # Get the AI response

    system_msg = chat_history[0]
    windowed_context = [system_msg] + chat_history[-6:]
    response = llm.invoke(windowed_context)
    ai_text = response.content

    print(f"Comedian: {ai_text}")

    # Add the AI's response to history so it remembers its own jokes!
    chat_history.append(AIMessage(content=ai_text))

--- Science Comedian Bot (Type 'exit' to stop) ---
You: The secret code is 4332
Comedian: Alright, alright, settle down folks! We've got a real brain-buster tonight, a code that's gonna make you scratch your heads and maybe even… *gasp*… think!

The secret code is... **4332**!

(Pauses for dramatic effect, maybe a little cough)

Now, I know what you're thinking. "Is that the number of times I've forgotten my password this week?" Probably. "Is that the winning lottery numbers for a very, very small lottery?" Highly unlikely.

But no, my friends, this is a *science* code! And if you're not a scientist, you might be thinking, "What does 4332 even *mean*?"

Well, let me tell you, it's a number that's got more layers than a perfectly baked onion… or a poorly explained quantum entanglement.

Let's break it down, shall we?

*   **The 4:** That's the number of fundamental forces in the universe. Gravity, electromagnetism, and the strong and weak nuclear forces. You know, the stuff that keeps y